In [1]:
import torch
import sys

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'MPS' if torch.backends.mps.is_available() else 'CPU'}")

Python: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]
PyTorch: 2.9.0
Device: MPS


In [6]:
import torch
import os
import time
import glob
import json
import shutil
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ultralytics import YOLO
from thop import profile
from pathlib import Path

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: mps


# YOLOv8 Training

Train YOLOv8n model on the dental X-ray dataset.


In [7]:
# Setup paths
project_root = Path('..').resolve() if Path.cwd().name == 'notebooks' else Path('.').resolve()
processed = project_root / 'data' / 'processed'
models_dir = project_root / 'models'
models_dir.mkdir(parents=True, exist_ok=True)

# Locate dataset
data_yaml_path = processed / 'dataset.yaml'
if not data_yaml_path.exists():
    raise FileNotFoundError("Dataset not found! Run 01_prepare_dataset.ipynb first.")

data_yaml_path = str(data_yaml_path)
test_images_path = Path(data_yaml_path).parent / "test" / "images"

print(f"✓ Data config: {data_yaml_path}")
print(f"✓ Models dir: {models_dir}")

✓ Data config: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/dataset.yaml
✓ Models dir: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/models


## Train Model


In [8]:
model_v8 = YOLO('yolov8n.pt')

print("Training YOLOv8n...")
start_time_v8 = time.time()
results_v8 = model_v8.train(
    data=data_yaml_path,
    epochs=1,  # Change to 50 for production
    imgsz=640,
    batch=8,
    name='yolov8n_dental',
    device=device,
    verbose=True,  # Shows progress bar with epochs
    plots=False,   # Disable plot generation
    save=True
)
train_time_v8 = time.time() - start_time_v8

path_v8_best_weights = results_v8.save_dir / 'weights' / 'best.pt'
print(f"\n✓ Training complete: {train_time_v8:.2f}s")
print(f"✓ Best weights: {path_v8_best_weights}")

Training YOLOv8n...
Ultralytics 8.3.63 🚀 Python-3.13.5 torch-2.9.0 MPS (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/dataset.yaml, epochs=1, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=yolov8n_dental4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_t

train: Scanning /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/train/labels.cache... 58 images, 0 backgrounds, 0 corrupt: 100%|██████████| 58/58 [00:00<?, ?it/s]
/Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
train: Scanning /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/train/labels.cache... 58 images, 0 backgrounds, 0 corrupt: 100%|██████████| 58/58 [00:00<?, ?it/s]
/Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov8n_dental4
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov8n_dental4
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  


        1/1      3.58G      1.499      4.158      1.282        141        640: 100%|██████████| 8/8 [00:11<00:00,  1.43s/it]
        1/1      3.58G      1.499      4.158      1.282        141        640: 100%|██████████| 8/8 [00:11<00:00,  1.43s/it]/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.15it/s]

                   all         17        499          0          0          0          0



1 epochs completed in 0.004 hours.
Optimizer stripped from /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov8n_dental4/weights/last.pt, 6.2MB
Optimizer stripped from /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov8n_dental4/weights/last.pt, 6.2MB
Optimizer stripped from /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov8n_dental4/weights/best.pt, 6.2MB

Validating /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov8n_dental4/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.13.5 torch-2.9.0 MPS (Apple M1)
Optimizer stripped from /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov8n_dental4/weights/best.pt, 6.2MB

Validating /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/dete

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.37it/s]

                   all         17        499          0          0          0          0
Speed: 1.9ms preprocess, 14.5ms inference, 0.0ms loss, 5.4ms postprocess per image
Speed: 1.9ms preprocess, 14.5ms inference, 0.0ms loss, 5.4ms postprocess per image



✓ Training complete: 16.67s
✓ Best weights: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov8n_dental4/weights/best.pt


## Profile Model


In [10]:
print("Profiling model...")
model_v8_trained = YOLO(path_v8_best_weights)

# Move to CPU for THOP compatibility
model_cpu = model_v8_trained.model.cpu()
dummy_input = torch.randn(1, 3, 640, 640)

flops_v8, params_v8 = profile(model_cpu, inputs=(dummy_input,), verbose=False)
print(f"✓ Parameters: {params_v8/1e6:.2f}M | FLOPs: {flops_v8/1e9:.2f}G")

Profiling model...
✓ Parameters: 3.01M | FLOPs: 4.10G
✓ Parameters: 3.01M | FLOPs: 4.10G


## Measure Inference Time


In [11]:
print("Measuring inference time...")
# Reload model on correct device for inference
model_v8_trained = YOLO(path_v8_best_weights)

test_images = list(test_images_path.glob('*.jpg')) + list(test_images_path.glob('*.png'))

def get_avg_inference_time(model, image_list):
    if len(image_list) == 0:
        return 0
    _ = model(image_list[0], verbose=False)  # Warm-up
    total_time = 0
    for img in image_list:
        start = time.perf_counter()
        _ = model(img, verbose=False)
        total_time += time.perf_counter() - start
    return (total_time / len(image_list)) * 1000

avg_inf_time_v8 = get_avg_inference_time(model_v8_trained, test_images[:10])
print(f"✓ Avg inference: {avg_inf_time_v8:.2f}ms/image ({len(test_images)} test images)")

Measuring inference time...
✓ Avg inference: 65.57ms/image (19 test images)
✓ Avg inference: 65.57ms/image (19 test images)


In [12]:
print("Evaluating on test set...")
metrics_v8 = model_v8_trained.val(split='test', data=data_yaml_path, verbose=False)
map50_v8 = metrics_v8.box.map50
map50_95_v8 = metrics_v8.box.map

print(f"✓ mAP@50: {map50_v8:.4f} | mAP@50-95: {map50_95_v8:.4f}")

Evaluating on test set...
Ultralytics 8.3.63 🚀 Python-3.13.5 torch-2.9.0 CPU (Apple M1)


val: Scanning /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/test/labels.cache... 19 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]
/Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/test/labels.cache... 19 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]
/Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
    

                   all         19        551          0          0          0          0


Speed: 1.2ms preprocess, 118.8ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/val
✓ mAP@50: 0.0000 | mAP@50-95: 0.0000
Results saved to /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/val
✓ mAP@50: 0.0000 | mAP@50-95: 0.0000


## Save Results


In [14]:
print("Saving model and metrics...")
shutil.copy(path_v8_best_weights, models_dir / 'yolov8_best.pt')

metrics_data = {
    'model': 'YOLOv8n',
    'training_time': train_time_v8,
    'params': params_v8 / 1e6,
    'inference_time_ms': avg_inf_time_v8,
    'map50': float(map50_v8),
    'map50_95': float(map50_95_v8)
}

with open(models_dir / 'yolov8_metrics.json', 'w') as f:
    json.dump(metrics_data, f, indent=2)

print(f"\n{'='*60}")
print(f"✓ Model saved: {models_dir}/yolov8_best.pt")
print(f"✓ Metrics saved: {models_dir}/yolov8_metrics.json")
print(f"{'='*60}")

Saving model and metrics...

✓ Model saved: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/models/yolov8_best.pt
✓ Metrics saved: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/models/yolov8_metrics.json


## Visualize Predictions


In [15]:
print("Visualizing sample predictions...")
sample_images = test_images[:3]
fig, axes = plt.subplots(1, len(sample_images), figsize=(15, 5))
if len(sample_images) == 1:
    axes = [axes]

for idx, img_path in enumerate(sample_images):
    results = model_v8_trained(img_path, verbose=False)
    plotted = results[0].plot()[..., ::-1]  # BGR to RGB
    axes[idx].imshow(plotted)
    axes[idx].axis('off')
    axes[idx].set_title(f"Sample {idx+1}")

plt.tight_layout()
plt.show()
print("\n✅ YOLOv8 training complete!")

Visualizing sample predictions...


<Figure size 1500x500 with 3 Axes>


✅ YOLOv8 training complete!
